# QM based correction of auranofin MM potential

This work is application of the force field correction technique published in https://doi.org/10.1021/acs.jcim.1c00651 on auranofin.
The QM methods were updated (mostly to handl gold properly) according to https://pubs.acs.org/doi/full/10.1021/ct400980w (thanks, M.Krupička).
The computational protocol is:
* Identify twistable torsions in the molecule
* Run replica exchange metadynamics, each replica having one of the torsions as its CV; this is believed to make quick, exhaustive, and reasonably realistic sampling of the conformational space
* Cluster the resulting trajectories to end up with at most few dozens of representative samples
* Run QM geometry optimization of all these samples while keeping the twistable torsions constrained
* Record final single point energies of the results
* Further relax the structures (with constrained torsions) in MM and record the energies
* The difference of the QM and MM energies can be used to add bias potential to MD (e.g. with https://www.plumed.org/doc-v2.8/user-doc/html/_p_r_o_p_e_r_t_y_m_a_p.html, based on https://doi.org/10.1063/1.3660208) to get a more realistic potential

The whole calculation is implemented in a fork of PMCVFF service: 
https://gitlab.ics.muni.cz/3086/magicforcefield-pipeline/-/blob/auranofin/auranofin.ipynb
and it requires the environment (e.g. https://pmcvff-correction.cerit-sc.cz) up and running.

This notebooks visualizes the resulting structures and gives elemetary insight to the energies.
Right now it is based on intermediate results (QM optimization of all the clusters has not converged) therefore it is illustrative only.




In [ ]:
import mdtraj as md
import nglview as nv
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# energies extracted from Orca/b2plyp calculation
# normalized to start from zero

b2plyp = {'outCluster11': 8.59025738733933,
 'outCluster12': 21.37990892940907,
 'outCluster8': 352.5826396459286,
 'outCluster26': 56.19611285873913,
 'outCluster4': 13.322411100455696,
 'outCluster15': 0.0,
 'outCluster1': 35.1602939508531,
 'outCluster22': 99.99590873653148,
 'outCluster17': 29.84600896256199,
 'outCluster7': 67.74622991454132,
 'outCluster20': 15.910550934496172,
 'outCluster24': 23.177356136004796,
 'outCluster23': 26.92110944118104,
 'outCluster2': 29.16100594583887,
 'outCluster27': 144.7391352797871,
 'outCluster14': 47.20687778861149,
 'outCluster18': 12.632784324504971}

In [ ]:
# energies of the output of b2plyp after constrained relaxation in MM 

mm = {'outCluster11': 1962.906875,
 'outCluster12': 1320.348037,
 'outCluster8': 1766.802993,
 'outCluster26': 885.243545,
 'outCluster4': 680.751907,
 'outCluster15': 682.627334,
 'outCluster1': 741.457229,
 'outCluster22': 912.0929100000001,
 'outCluster17': 13.167816000000016,
 'outCluster7': 552.935501,
 'outCluster20': 1610.843277,
 'outCluster24': 611.733535,
 'outCluster23': 488.26771499999995,
 'outCluster2': 1107.932876,
 'outCluster27': 39.378615999999994,
 'outCluster14': 1152.737808,
 'outCluster18': 0.0}

In [ ]:
# energy deltas of the MM constrained relaxation

mmdelta = dict()
for c in mm.keys():
    xvg = np.loadtxt(f'{c}.xvg',comments=list('@#'))
    mmdelta[c] = xvg[-1,1]-xvg[0,1]
mmdelta

In [ ]:
"""
Align (on the C-S-Au-P torsion) and display the structures

Coloring:
    - yellow: input structure for b2plyp optimization
    - blue: output of b2plyp optimization
    - green: further constrained relaxation in MM
"""

trs_cl, trs_qm, trs_mm = [],[],[]
ref = md.load_pdb('outCluster0.pdb')

rmsd_clqm, rmsd_qmmm = [],[]

idx = [9,10,43,51]

for c in mm.keys():
    trqm = md.load(f'{c}_opt.pdb')
    trcl = md.load(f'{c}.pdb')
    trmm = md.load(f'{c}_after_em1.gro')
    
    rmsd_clqm.append(md.rmsd(trqm,trcl[0])[0])
    rmsd_qmmm.append(md.rmsd(trmm,trqm[0])[0])
    
    trqm.superpose(ref,atom_indices=idx)
    trs_qm.append(trqm)
    trcl.superpose(ref,atom_indices=idx)
    trs_cl.append(trcl)
    trmm.superpose(ref,atom_indices=idx)
    trs_mm.append(trmm)
    

    
trqm = md.join(trs_qm)
trcl = md.join(trs_cl)
trmm = md.join(trs_mm)

In [ ]:

r=.5
v = nv.NGLWidget()
v.clear()

cqm = v.add_trajectory(trqm)
cqm.clear()
cqm.add_licorice(color = 'blue')
cqm.add_representation('spacefill',selection=[10,43,51],radius=r)

ccl = v.add_trajectory(trcl)
ccl.clear()
ccl.add_licorice(color = 'yellow')
ccl.add_representation('spacefill',selection=[10,43,51],radius=r)

cmm = v.add_trajectory(trmm)
cmm.clear()
cmm.add_licorice(color = 'green')
cmm.add_representation('spacefill',selection=[10,43,51],radius=r)


v

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(b2plyp.values(),label='b2plyp')
plt.plot(np.array(list(mm.values()))-np.array(list(mmdelta.values())),label='mm after b2plyp')
plt.plot(mm.values(),label='mm final')
plt.ylabel('kJ/mol')
plt.xlabel('conformation')
plt.xticks(range(len(mm)),labels=[ f'{k} : {v}' for k,v in enumerate(mm.keys())],rotation=60)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(rmsd_clqm,label='RMSD between b2plyp input-output')
plt.plot(rmsd_qmmm,label='RMSD between MM relaxation input-output')
plt.ylabel('A')
plt.xlabel('conformation')
plt.xticks(range(len(mm)),labels=[ f'{k} : {v}' for k,v in enumerate(mm.keys())],rotation=60)
plt.legend()
plt.show()